In [30]:
import json
import pandas as pd
from pathlib import Path
from pprint import pprint
import datetime
import time
from scipy import stats
import numpy as np
import matplotlib as plt

import pandas
import researchpy as rp
import seaborn as sns

import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp

%matplotlib inline

pd.set_option('display.max_rows', 10000)

In [2]:
TRAINING_ROUND = 'Training_Round'
EPOCHS = 'Epochs'
SCENARIO = 'Scenario'
EMBEDDING = 'Embedding'
TAGGING_SCHEME = 'Tagging_Scheme'
CAP_DIM = 'Capitalisation_Dim'
LOWER = 'Lower'
CHAR_DIM = 'Char_LSTM_Dim'
WORD_DIM = 'Word_LSTM_Dim'
PRECISION = 'Precision'
RECALL = 'Recall'
F1_SCORE = 'F1_Score'

In [3]:
columns = [TRAINING_ROUND, EPOCHS, SCENARIO, EMBEDDING, TAGGING_SCHEME, CAP_DIM, LOWER, CHAR_DIM, WORD_DIM, PRECISION, RECALL, F1_SCORE]

In [4]:
columns

['Training_Round',
 'Epochs',
 'Scenario',
 'Embedding',
 'Tagging_Scheme',
 'Capitalisation_Dim',
 'Lower',
 'Char_LSTM_Dim',
 'Word_LSTM_Dim',
 'Precision',
 'Recall',
 'F1_Score']

In [5]:
pd.set_option('display.max_columns', 1000)
training_data_df = pd.read_csv('resultados_sem_outliers.csv')
training_data_df

,Unnamed: 0,Training_Round,Epochs,Scenario,Embedding,Tagging_Scheme,Capitalisation_Dim,Lower,Char_LSTM_Dim,Word_LSTM_Dim,Precision,Recall,F1_Score
0,320,0,5,selective,FastText,IOB2,False,False,25,100,58.83,56.51,57.64
1,352,1,5,selective,FastText,IOB2,False,False,25,100,64.86,50.77,56.96
2,384,2,5,selective,FastText,IOB2,False,False,25,100,63.22,51.77,56.93
3,416,3,5,selective,FastText,IOB2,False,False,25,100,59.05,56.76,57.88
4,448,4,5,selective,FastText,IOB2,False,False,25,100,58.87,55.11,56.93
5,480,5,5,selective,FastText,IOB2,False,False,25,100,59.32,48.73,53.50
6,512,6,5,selective,FastText,IOB2,False,False,25,100,59.65,50.99,54.98
7,544,7,5,selective,FastText,IOB2,False,False,25,100,63.90,50.59,56.47
8,576,8,5,selective,FastText,IOB2,False,False,25,100,59.73,56.47,58.05
9,608,9,5,selective,FastText,IOB2,False,False,25,100,62.26,55.36,58.61


In [6]:
training_data_df = training_data_df[training_data_df.Scenario == 'selective']
len(training_data_df)

1157

In [7]:
training_data_df.groupby([TAGGING_SCHEME]).describe()[F1_SCORE].sort_values(by='mean', ascending=False)

,count,mean,std,min,25%,50%,75%,max
Tagging_Scheme,,,,,,,,
IOB2,583.0,57.993756,5.794382,44.34,54.47,56.260,63.7650,68.26
IOBES,574.0,57.959739,5.849190,42.97,54.56,56.635,63.5975,67.99


In [8]:
training_data_df.groupby([EMBEDDING]).describe()[F1_SCORE].sort_values(by='mean', ascending=False)

,count,mean,std,min,25%,50%,75%,max
Embedding,,,,,,,,
Wang2Vec,286.0,61.274895,5.270895,51.42,56.2600,58.58,66.4775,68.26
FastText,280.0,60.694143,4.480110,53.42,56.4950,58.81,65.1750,66.90
Glove,287.0,58.839861,3.973881,50.18,55.1350,60.31,62.7100,64.89
Word2Vec,304.0,51.556678,3.243757,42.97,48.9725,51.92,54.5000,56.92


In [9]:
training_data_df.groupby([CAP_DIM]).describe()[F1_SCORE].sort_values(by='mean', ascending=False)

,count,mean,std,min,25%,50%,75%,max
Capitalisation_Dim,,,,,,,,
True,583.0,58.127118,5.861585,43.81,54.5750,56.610,63.84,68.26
False,574.0,57.824286,5.776824,42.97,54.4625,56.275,63.43,68.07


In [10]:
training_data_df.groupby([CHAR_DIM]).describe()[F1_SCORE].sort_values(by='mean', ascending=False)

,count,mean,std,min,25%,50%,75%,max
Char_LSTM_Dim,,,,,,,,
50,571.0,58.048319,5.750252,45.04,54.65,56.530,63.6900,68.07
25,586.0,57.907270,5.889573,42.97,54.34,56.425,63.5925,68.26


In [11]:
training_data_df.groupby([WORD_DIM]).describe()[F1_SCORE].sort_values(by='mean', ascending=False)

,count,mean,std,min,25%,50%,75%,max
Word_LSTM_Dim,,,,,,,,
100,582.0,58.150722,5.768651,42.97,54.7075,56.645,63.860,68.26
200,575.0,57.800922,5.869582,43.81,54.2800,56.360,63.565,68.07


In [12]:
training_data_df.groupby([LOWER]).describe()[F1_SCORE].sort_values(by='mean', ascending=False)

,count,mean,std,min,25%,50%,75%,max
Lower,,,,,,,,
True,581.0,60.439897,7.293584,42.97,51.50,63.61,65.860,68.26
False,576.0,55.492483,1.451393,50.18,54.57,55.58,56.535,59.56


In [13]:
training_data_df.groupby([EMBEDDING, TAGGING_SCHEME, CAP_DIM, WORD_DIM, CHAR_DIM, LOWER]).describe()[F1_SCORE].sort_values(by='mean', ascending=False)

count  \
Embedding Tagging_Scheme Capitalisation_Dim Word_LSTM_Dim Char_LSTM_Dim Lower          
Wang2Vec  IOB2           False              100           50            True    10.0   
                         True               100           25            True     9.0   
                         False              200           50            True     6.0   
          IOBES          True               200           25            True    10.0   
                                            100           50            True     9.0   
          IOB2           True               100           50            True    10.0   
                                            200           50            True     9.0   
          IOBES          True               100           25            True    10.0   
          IOB2           True               200           25            True     7.0   
          IOBES          False              100           50            True     7.0   
                         True               200           50            True    10.0   
                         False              200           50            True     9.0   
                                            100           25            True     9.0   
          IOB2           False              200           25            True     9.0   
                                            100           25            True     8.0   
          IOBES          False              200           25            True     9.0   
FastText  IOB2           True               100           25            True    10.0   
                                                          50            True     9.0   
          IOBES          False              100           25            True     9.0   
                         True               200           50            True     8.0   
          IOB2           True               200           25            True     9.0   
                         False              100           25            True    10.0   
          IOBES          True               100           25            True     7.0   
                         False              100           50            True     8.0   
          IOB2           False              100           50            True     8.0   
          IOBES          False              200           25            True     9.0   
                         True               200           25            True    10.0   
          IOB2           False              200           50            True    10.0   
                                                          25            True     8.0   
                         True               200           50            True     9.0   
...                                                                              ...   
Word2Vec  IOB2           True               200           50            False    8.0   
                         False              100           25            False    9.0   
          IOBES          True               200           25            False   10.0   
Glove     IOB2           True               100           25            False    9.0   
Word2Vec  IOBES          True               100           50            False   10.0   
                                                          25            False   10.0   
                         False              100           25            False   10.0   
Glove     IOB2           False              200           50            False   10.0   
Word2Vec  IOB2           False              200           50            False   10.0   
Glove     IOB2           False              200           25            False    9.0   
Word2Vec  IOBES          True               200           50            False    8.0   
          IOB2           False              200           25            False   10.0   
          IOBES          False              200           25            False   10.0   
          IOB2           True  

In [14]:
training_data_df.groupby([TRAINING_ROUND]).describe()[F1_SCORE].sort_values(by='mean', ascending=False)

,count,mean,std,min,25%,50%,75%,max
Training_Round,,,,,,,,
2,121.0,58.226942,5.917851,44.08,54.8800,56.530,64.2700,68.07
9,115.0,58.137391,5.631239,46.94,54.7500,56.530,63.5100,67.87
5,111.0,58.067748,5.917702,47.59,54.6100,56.440,63.9900,67.86
6,119.0,58.048824,5.825342,45.04,54.9200,56.710,63.5750,67.74
4,114.0,58.021404,5.518434,47.28,54.5975,56.720,63.6700,68.26
8,116.0,57.981207,5.754974,46.61,54.2375,56.295,63.7225,67.73
1,118.0,57.963644,5.883625,46.26,54.5700,56.040,63.4050,67.99
3,113.0,57.950973,6.073921,44.33,54.3100,56.290,63.9000,67.79
0,121.0,57.923636,5.849924,42.97,54.5700,56.590,63.1800,67.57


In [15]:
rp.summary_cont(training_data_df[F1_SCORE])

,Variable,N,Mean,SD,SE,95% Conf.,Interval
0,F1_Score,1157.0,57.97688,5.819143,0.171077,57.641223,58.312537


In [16]:
rp.summary_cont(training_data_df.groupby(EMBEDDING))[F1_SCORE]

,N,Mean,SD,SE,95% Conf.,Interval
Embedding,,,,,,
FastText,280,60.694143,4.480110,0.267738,60.169377,61.218909
Glove,287,58.839861,3.973881,0.234571,58.380102,59.299619
Wang2Vec,286,61.274895,5.270895,0.311675,60.664013,61.885777
Word2Vec,304,51.556678,3.243757,0.186042,51.192035,51.921320


In [17]:
print(F1_SCORE, EMBEDDING, TAGGING_SCHEME, CAP_DIM, CHAR_DIM, WORD_DIM, LOWER)

F1_Score Embedding Tagging_Scheme Capitalisation_Dim Char_LSTM_Dim Word_LSTM_Dim Lower


In [33]:
# Fits the model with the interaction term
# This will also automatically include the main effects for each factor
model = ols('F1_Score ~ C(Embedding)*C(Tagging_Scheme)*C(Capitalisation_Dim)*C(Char_LSTM_Dim)*C(Word_LSTM_Dim)*C(Lower)', training_data_df).fit()

# Seeing if the overall model is significant
print(f"Overall model F({model.df_model: .0f},{model.df_resid: .0f}) = {model.fvalue: .3f}, p = {model.f_pvalue: .20f}")

Overall model F( 127, 1029) =  202.677, p =  0.00000000000000000000


In [34]:
model.mse_model, model.mse_resid, model.mse_total, model.f_pvalue

(296.3797916697224, 1.4623246785675723, 33.86242702102144, 0.0)

In [35]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               F1_Score   R-squared:                       0.962
Model:                            OLS   Adj. R-squared:                  0.957
Method:                 Least Squares   F-statistic:                     202.7
Date:                Wed, 10 Apr 2019   Prob (F-statistic):               0.00
Time:                        11:59:55   Log-Likelihood:                -1793.7
No. Observations:                1157   AIC:                             3843.
Df Residuals:                    1029   BIC:                             4490.
Df Model:                         127                                         
Covariance Type:            nonrobust                                         
=====================================================================================================================================================================================================================
                                                                                                                                                        coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                                                                                                            56.7950      0.382    148.521      0.000      56.045      57.545
C(Embedding)[T.Glove]                                                                                                                                -1.1490      0.541     -2.125      0.034      -2.210      -0.088
C(Embedding)[T.Wang2Vec]                                                                                                                             -1.1730      0.541     -2.169      0.030      -2.234      -0.112
C(Embedding)[T.Word2Vec]                                                                                                                             -2.2372      0.556     -4.027      0.000      -3.327      -1.147
C(Tagging_Scheme)[T.IOBES]                                                                                                                           -0.8836      0.596     -1.483      0.138      -2.053       0.286
C(Capitalisation_Dim)[T.True]                                                                                                                        -0.5390      0.541     -0.997      0.319      -1.600       0.522
C(Char_LSTM_Dim)[T.50]                                                                                                                               -0.4817      0.556     -0.867      0.386      -1.572       0.609
C(Word_LSTM_Dim)[T.200]                                                                                                                              -1.3887      0.574     -2.421      0.016      -2.514      -0.263
C(Lower)[T.True]                                                                                                                                      8.4640      0.541     15.651      0.000       7.403       9.525
C(Embedding)[T.Glove]:C(Tagging_Scheme)[T.IOBES]                                                                                                      0.3526      0.805      0.438      0.661      -1.227       1.932
C(Embedding)[T.Wang2Vec]:C(Tagging_Scheme)[T.IOBES]                                                                                                   2.5178      0.827      3.044      0.002       0.895       4.141
C(Embedding)[T.Word2Vec]:C(Tagging_Scheme)[T.IOBES]                                    

In [36]:
res = sm.stats.anova_lm(model, typ= 2)

In [37]:
res

,sum_sq,df,F,PR(>F)
C(Embedding),18097.602151,3.0,4125.304140,0.000000
C(Tagging_Scheme),1.857111,1.0,1.269972,0.260035
C(Capitalisation_Dim),7.036934,1.0,4.812156,0.028483
C(Char_LSTM_Dim),9.587457,1.0,6.556312,0.010593
C(Word_LSTM_Dim),35.019032,1.0,23.947508,0.000001
C(Lower),7231.789826,1.0,4945.406401,0.000000
C(Embedding):C(Tagging_Scheme),9.464230,3.0,2.157348,0.091458
C(Embedding):C(Capitalisation_Dim),9.726866,3.0,2.217215,0.084545
C(Tagging_Scheme):C(Capitalisation_Dim),0.050563,1.0,0.034577,0.852521
C(Embedding):C(Char_LSTM_Dim),20.452179,3.0,4.662024,0.003046


In [23]:
# Calculating effect size
def anova_table(aov):
    aov['mean_sq'] = aov[:]['sum_sq']/aov[:]['df']
    
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*aov['mean_sq'][-1]))/(sum(aov['sum_sq'])+aov['mean_sq'][-1])
    
    cols = ['sum_sq', 'mean_sq', 'df', 'F', 'PR(>F)', 'eta_sq', 'omega_sq']
    aov = aov[cols]
    return aov

anova_table(res)

,sum_sq,mean_sq,df,F,PR(>F),eta_sq,omega_sq
C(Embedding),18097.602151,6032.534050,3.0,4125.304140,0.000000,4.620814e-01,4.619521e-01
C(Tagging_Scheme),1.857111,1.857111,1.0,1.269972,0.260035,4.741713e-05,1.007960e-05
C(Capitalisation_Dim),7.036934,7.036934,1.0,4.812156,0.028483,1.796722e-04,1.423297e-04
C(Char_LSTM_Dim),9.587457,9.587457,1.0,6.556312,0.010593,2.447941e-04,2.074492e-04
C(Word_LSTM_Dim),35.019032,35.019032,1.0,23.947508,0.000001,8.941318e-04,8.567627e-04
C(Lower),7231.789826,7231.789826,1.0,4945.406401,0.000000,1.846474e-01,1.846032e-01
C(Embedding):C(Tagging_Scheme),9.464230,3.154743,3.0,2.157348,0.091458,2.416477e-04,1.296314e-04
C(Embedding):C(Capitalisation_Dim),9.726866,3.242289,3.0,2.217215,0.084545,2.483535e-04,1.363370e-04
C(Tagging_Scheme):C(Capitalisation_Dim),0.050563,0.050563,1.0,0.034577,0.852521,1.291019e-06,-3.604479e-05
C(Embedding):C(Char_LSTM_Dim),20.452179,6.817393,3.0,4.662024,0.003046,5.222002e-04,4.101734e-04


In [24]:
mc = statsmodels.stats.multicomp.MultiComparison(training_data_df[F1_SCORE], training_data_df[EMBEDDING])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD,FWER=0.05
 group1   group2  meandiff  lower    upper  reject
--------------------------------------------------
FastText  Glove   -1.8543  -2.7815  -0.9271  True 
FastText Wang2Vec  0.5808  -0.3472   1.5087 False 
FastText Word2Vec -9.1375  -10.0517 -8.2232  True 
 Glove   Wang2Vec  2.435    1.5128   3.3573  True 
 Glove   Word2Vec -7.2832  -8.1916  -6.3747  True 
Wang2Vec Word2Vec -9.7182  -10.6275 -8.8089  True 
--------------------------------------------------


In [25]:
mc = statsmodels.stats.multicomp.MultiComparison(training_data_df[F1_SCORE], training_data_df[CAP_DIM])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD,FWER=0.05
group1 group2 meandiff  lower  upper  reject
--------------------------------------------
False   True   0.3028  -0.3686 0.9742 False 
--------------------------------------------


In [26]:
mc = statsmodels.stats.multicomp.MultiComparison(training_data_df[F1_SCORE], training_data_df[LOWER])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD,FWER=0.05
group1 group2 meandiff lower  upper  reject
-------------------------------------------
False   True   4.9474  4.3396 5.5553  True 
-------------------------------------------


In [27]:
mc = statsmodels.stats.multicomp.MultiComparison(training_data_df[F1_SCORE], training_data_df[TAGGING_SCHEME])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD,FWER=0.05
group1 group2 meandiff  lower  upper  reject
--------------------------------------------
 IOB2  IOBES   -0.034  -0.7056 0.6376 False 
--------------------------------------------


In [28]:
mc = statsmodels.stats.multicomp.MultiComparison(training_data_df[F1_SCORE], training_data_df[CHAR_DIM])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD,FWER=0.05
group1 group2 meandiff  lower  upper  reject
--------------------------------------------
  25     50    0.141   -0.5306 0.8127 False 
--------------------------------------------


In [29]:
mc = statsmodels.stats.multicomp.MultiComparison(training_data_df[F1_SCORE], training_data_df[WORD_DIM])
mc_results = mc.tukeyhsd()
print(mc_results)

Multiple Comparison of Means - Tukey HSD,FWER=0.05
group1 group2 meandiff  lower  upper  reject
--------------------------------------------
 100    200   -0.3498  -1.0211 0.3215 False 
--------------------------------------------
